In [ ]:
import matplotlib.pyplot as plt
# import scipy.stats as st
import seaborn as sns
import torch.nn as nn
import pandas as pd
import numpy as np

import itertools
import warnings
import torch

from torch.utils.data import DataLoader, TensorDataset, Dataset
from copy import deepcopy

from wave_generator import WaveGen

In [ ]:
large_noise_wave = WaveGen(size=1_000).linear_phase().cos().amp(.5).t_noise(std=.01, dof=5).sample(1).samples.squeeze()
small_noise_wave = WaveGen(size=1_000).linear_phase().cos().amp(.5).t_noise(std=.05, dof=5).sample(1).samples.squeeze()
noise_wave = np.r_[large_noise_wave, small_noise_wave]

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(noise_wave)
plt.show()

In [116]:
class SineBinarySeparator(nn.Module):
    def __init__(self, input_dim=2000, output_dim=2000, dropout_p=.5):
        super(SineBinarySeparator, self).__init__()
        self.input = nn.Linear(input_dim, output_dim)
        self.activation = nn.ReLU()
        self.hidden = nn.Linear(output_dim, output_dim)
        self.dropout = nn.Dropout(p=dropout_p)
        self.hidden2 = nn.Linear(output_dim, input_dim)
        self.output = nn.Linear(input_dim, 1)
        return
        
    def forward(self, x):
        x = self.input(x)
#         x = self.activation(x)
        x = torch.where(x < 0, 0, x)
        x = self.hidden(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.hidden2(x)
#         with torch.no_grad():
        hyperbolic_x = torch.tanh(x)
        return hyperbolic_x
#         scalar_x = self.output(x)
#         return hyperbolic_x, scalar_x
    
    def stepwise_loss(self, binary_x, x):
        with torch.no_grad():
            above = torch.where(binary_x < 0, 0, x)
            below = torch.where(binary_x < 0, x, 0)
            return above.var() - below.var()

    def multiple_loss(self, hyperbolic_x, x):
        hyper_detach = hyperbolic_x.detach()
        x_detach = x.detach()
#         zero_tensor = torch.zeros_like(hyperbolic_x)
        zero_up = torch.where(hyperbolic_x < 0, 1, 0)
        zero_down = torch.where(hyperbolic_x >= 0, 1, 0)
        up_count = torch.count_nonzero(zero_up)
        down_count = torch.count_nonzero(zero_down)
        x = x - x.mean()
        x = torch.square(x)
        up_x = x * zero_up
        up_x = up_x.sum() / up_count
        down_x = x * zero_down
        down_x = down_x.sum() / down_count
        return 
        
    def configure_optimizer(self, **kwargs):
        optimizer = torch.optim.RMSprop(self.parameters(), **kwargs)
        return optimizer
    
    def train(self, x, epochs=1000):
        if not hasattr(self, 'loss_items'):
            self.loss_items = list()
        optimizer = self.configure_optimizer()
        criterion = nn.MSELoss()
        for epoch in range(epochs):
            optimizer.zero_grad()
            hyperbolic_x = self.forward(x)
            loss = self.multiple_loss(self, hyperbolic_x, x)
#             loss = self.loss(hyperbolic_x.detach(), x)
#             loss = criterion(hyperbolic_x, x)
            loss.backward()
            optimizer.step()
            self.loss_items.append(loss.item())
        return
            
    def plot_loss(self, step=1, title=''):
        plt.figure(figsize=(16,8))
        sns.lineplot(self.loss_items[::step], dashes=False, palette='Spectral', label='Variance Difference Loss')
        plt.title(title)
        plt.legend()
        plt.show()
        return

In [117]:
wave_tensor = torch.from_numpy(noise_wave).float().clone()

model = SineBinarySeparator(input_dim=wave_tensor.shape[0]).float()
model.train(wave_tensor, epochs=10)
model.plot_loss()
model.train(wave_tensor, epochs=10)
model.plot_loss()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn